## Summarization

In this example, you will learn how to create a summarization app and evaluate + track it in TruLens

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/langchain/langchain_summarize.ipynb)

### Import libraries

In [ ]:
# ! pip install trulens_eval==0.11.0 langchain==0.0.283 langchain_community

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import OpenAI
from trulens.ext.instrument.langchain import Feedback
from trulens.ext.instrument.langchain import FeedbackMode
from trulens.ext.instrument.langchain import Query
from trulens.ext.instrument.langchain import Tru
from trulens.ext.instrument.langchain import TruChain
from trulens.external.provider import OpenAI

tru = Tru()

### Set API Keys

For this example, we need API keys for the Huggingface and OpenAI

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

In [ ]:
provider = OpenAI()

# Define a moderation feedback function using HuggingFace.
mod_not_hate = Feedback(provider.moderation_not_hate).on(
    text=Query.RecordInput[:].page_content
)


def wrap_chain_trulens(chain):
    return TruChain(
        chain,
        app_id="ChainOAI",
        feedbacks=[mod_not_hate],
        feedback_mode=FeedbackMode.WITH_APP,  # calls to TruChain will block until feedback is done evaluating
    )


def get_summary_model(text):
    """
    Produce summary chain, given input text.
    """

    llm = OpenAI(temperature=0, openai_api_key="")
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " "], chunk_size=8000, chunk_overlap=350
    )
    docs = text_splitter.create_documents([text])
    print(f"You now have {len(docs)} docs instead of 1 piece of text.")

    return docs, load_summarize_chain(llm=llm, chain_type="map_reduce")

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")
text = billsum["text"][0]

docs, chain = get_summary_model(text)

# use wrapped chain as context manager
with wrap_chain_trulens(chain) as recording:
    chain(docs)

### Run the TruLens dashboard

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(tru)